## Goal: Make anyone speak anything (LipSync)

* Github: https://github.com/Rudrabha/Wav2Lip
* Paper: https://arxiv.org/abs/2008.10010
*Original notebook: https://colab.research.google.com/drive/1tZpDWXz49W6wDcTprANRGLo2D_EbD5J8?usp=sharing




**Modded by: [justinjohn-03](https://github.com/justinjohn0306)**
and

**Modded by: [Tejesh-JP](https://github.com/Tejesh-JP)**





In [ ]:
#@title <h1>Step1: Setup Wav2Lip</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model
from IPython.display import HTML, clear_output
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11

from IPython.display import clear_output

clear_output()
print("All set and ready!")

# LipSync on Your Video File

In [ ]:
import os
import shutil
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp


def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

# Mount Google Drive if it's not already mounted
if not os.path.isdir("/content/drive/MyDrive"):
    drive.mount('/content/drive', force_remount=True)

#@markdown ### Select an uploading method
upload_method = "Custom Path" #@param ["Upload", "Custom Path"]

# remove previous input video
if os.path.isfile('/content/sample_data/input_vid.mp4'):
    os.remove('/content/sample_data/input_vid.mp4')

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, '/content/sample_data/input_vid.mp4')
    PATH_TO_YOUR_VIDEO = '/content/sample_data/input_vid.mp4'

elif upload_method == 'Custom Path':
    #@markdown ``Add the full path to your video on your Gdrive `` 👇
    PATH_TO_YOUR_VIDEO = '/content/drive/MyDrive/Wav2Lip/video.mp4' #@param {type:"string"}
    if not os.path.isfile(PATH_TO_YOUR_VIDEO):
        print("ERROR: File not found!")
        raise SystemExit(0)

#@markdown ___

clear_output()
print("Input Video")
showVideo(PATH_TO_YOUR_VIDEO)


In [ ]:
#@markdown ### Video transformation
#@markdown * Removing audio from video
#@markdown * Removing unwanted frames which does not contain any face
#@markdown * Removing unwanted frames where face is partially hidden



# Install required modules
!pip install opencv-python moviepy tqdm

import cv2
from moviepy.editor import VideoFileClip
from tqdm import tqdm

# Function to detect faces in a frame using OpenCV Haarcascades
def detect_faces(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6)
    return faces

# Function to process the video and extract frames with faces using Haarcascades
def process_video(input_path, output_path, frame_skip=50):
    # Load the pre-trained Haarcascades face detection classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    clip = VideoFileClip(input_path)

    # Store the frames with faces
    filtered_frames = []

    for i, frame in enumerate(tqdm(clip.iter_frames(fps=clip.fps), total=int(clip.duration * clip.fps))):
        if i % frame_skip == 0:
            faces = detect_faces(frame, face_cascade)
            if len(faces) == 1:
                # Only add frames with detected faces to the filtered_frames list
                filtered_frames.append(frame)

    # Create a new video clip with the filtered frames
    filtered_clip = VideoFileClip(input_path).set_audio(clip.audio)
    filtered_clip = filtered_clip.set_duration(len(filtered_frames) / clip.fps)
    filtered_clip = filtered_clip.set_fps(clip.fps)

    # Write the filtered video to the output path
    filtered_clip.write_videofile(output_path, audio=False, codec="libx264", preset="ultrafast", threads=4, write_logfile=True)

if __name__ == "__main__":
    input_video_path = "/content/sample_data/input_vid.mp4"
    output_video_path = "/content/sample_data/input_vid1.mp4"

    process_video(input_video_path, output_video_path, frame_skip=50)


In [ ]:
#@title STEP3: Select Audio (Record, Upload from local drive or Gdrive)
import os
from IPython.display import Audio
from IPython.core.display import display

upload_method = 'Custom Path' #@param ['Custom Path']

#remove previous input audio
if os.path.isfile('/content/sample_data/input_audio.wav'):
    os.remove('/content/sample_data/input_audio.wav')

def displayAudio():
  display(Audio('/content/sample_data/input_audio.wav'))



from google.colab import drive
drive.mount('/content/drive')
#@markdown ``Add the full path to your audio on your Gdrive`` 👇
PATH_TO_YOUR_AUDIO = '/content/drive/MyDrive/Wav2Lip/audio.wav' #@param {type:"string"}

# Load audio with specified sampling rate
import librosa
audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

# Save audio with specified sampling rate
import soundfile as sf
sf.write('/content/sample_data/input_audio.wav', audio, sr, format='wav')

clear_output()
displayAudio()


In [ ]:
#@title STEP4: Start Crunching and Preview Output
#@markdown <b>Note: Only change these, if you have to</b>

%cd /content/Wav2Lip

# Set up paths and variables for the output file
output_file_path = '/content/Wav2Lip/results/result_voice.mp4'

# Delete existing output file before processing, if any
if os.path.exists(output_file_path):
    os.remove(output_file_path)

pad_top =  3#@param {type:"integer"}
pad_bottom =  3#@param {type:"integer"}
pad_left =  3#@param {type:"integer"}
pad_right =  3#@param {type:"integer"}
rescaleFactor =  2#@param {type:"integer"}
nosmooth = False #@param {type:"boolean"}
#@markdown ___
#@markdown Model selection:
use_hd_model = True #@param {type:"boolean"}
checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'


if nosmooth == False:
  !python inference.py --checkpoint_path $checkpoint_path --face "../sample_data/input_vid1.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !python inference.py --checkpoint_path $checkpoint_path --face "../sample_data/input_vid1.mp4" --audio "../sample_data/input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

#Preview output video
if os.path.exists(output_file_path):
    clear_output()
    print("Final Video Preview")
    print("Download this video from", output_file_path)
    showVideo(output_file_path)
else:
    print("Processing failed. Output video not found.")